In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au


In [2]:
input_text = au.read_txt_file_lines()
n_rows = len(input_text)
n_cols = len(input_text[0])
for ii in range(1, n_rows):
    assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

input has 140 rows and 140 cols


In [6]:
def expand_universe(input_text, expand=True):
    '''Turn into 2D np array. Expand all empty rows and col'''
    arr_universe = np.zeros((n_rows, n_cols), dtype=np.int8)
    for row in range(n_rows):
        for col in range(n_cols):
            if input_text[row][col] == '#':
                arr_universe[row, col] = 1
            else:
                arr_universe[row, col] = 0

    empty_rows = np.all(arr_universe == 0, axis=1)
    empty_cols = np.all(arr_universe == 0, axis=0)
    print(f'There are {np.sum(empty_rows)} empty rows and {np.sum(empty_cols)} empty cols')
    # print(f'Empty rows; {np.where(empty_rows)[0]}, empty cols: {np.where(empty_cols)[0]}')

    if expand:
        arr_universe = np.insert(arr_universe, np.where(empty_rows)[0], 0, axis=0)
        arr_universe = np.insert(arr_universe, np.where(empty_cols)[0], 0, axis=1)

    return arr_universe, (empty_rows, empty_cols)

arr_universe, _ = expand_universe(input_text)

There are 6 empty rows and 10 empty cols


In [5]:
def find_distance_all_pairs(arr_universe):
    inds_galaxy = np.where(arr_universe == 1)
    n_galaxies = len(inds_galaxy[0])
    total_distance = 0 
    n_pairs = 0
    for ii in range(n_galaxies):
        for jj in range(ii, n_galaxies):
            if ii != jj:
                n_pairs += 1
                dist = abs(inds_galaxy[0][ii] - inds_galaxy[0][jj]) + abs(inds_galaxy[1][ii] - inds_galaxy[1][jj])
                # print(f'Inds: {inds_galaxy[0][ii], inds_galaxy[1][ii]} and {inds_galaxy[0][jj], inds_galaxy[1][jj]}. Distance: {dist}')
                total_distance += dist
    print(f'There are {n_pairs} pairs of galaxies')
    return total_distance
total_dist = find_distance_all_pairs(arr_universe)
print(f'Total distance: {total_dist}')

There are 96580 pairs of galaxies
Total distance: 9918828


## part 2 


In [22]:
arr_universe, (empty_rows, empty_cols) = expand_universe(input_text, expand=False)
inds_empty_rows = np.where(empty_rows)[0]
inds_empty_cols = np.where(empty_cols)[0]

def find_distance_all_pairs_including_empty(arr_universe, inds_empty_rows, inds_empty_cols):
    inds_galaxy = np.where(arr_universe == 1)
    n_galaxies = len(inds_galaxy[0])
    total_distance = 0 
    dist_empty = 999999  # 1e6 - 1 because 1 is already added  for the original (empty) row/col 
    for ii in range(n_galaxies):
        for jj in range(ii, n_galaxies):
            if ii != jj:
                dist_init = abs(inds_galaxy[0][ii] - inds_galaxy[0][jj]) + abs(inds_galaxy[1][ii] - inds_galaxy[1][jj])
                row_min = np.minimum(inds_galaxy[0][ii], inds_galaxy[0][jj])
                row_max = np.maximum(inds_galaxy[0][ii], inds_galaxy[0][jj])
                col_min = np.minimum(inds_galaxy[1][ii], inds_galaxy[1][jj])
                col_max = np.maximum(inds_galaxy[1][ii], inds_galaxy[1][jj])
                n_empty = 0
                for row in inds_empty_rows:
                    if row in range(row_min, row_max):
                        n_empty += 1
                for col in inds_empty_cols:
                    if col in range(col_min, col_max):
                        n_empty += 1    
                dist = dist_init + n_empty * dist_empty
                total_distance += dist
    return total_distance

total_dist = find_distance_all_pairs_including_empty(arr_universe, inds_empty_rows, inds_empty_cols)
print(f'Total distance: {total_dist}')

There are 6 empty rows and 10 empty cols
Total distance: 692506533832
